In [ ]:
!pip install streamlit --quiet
!pip install streamlit pyngrok --quiet
!pip install catboost==1.2.7
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


*Voting* Regressor

In [ ]:
import joblib
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
# ✅ Load the data
data = pd.read_excel("/content/drive/MyDrive/육사 부식 잔반 최적화/current_processed_menu_data.xlsx")

# ✅ Select relevant columns and drop NaN values
data_subset = data[['Meal Type', 'Menu', 'Dessert', 'Event', 'leftovers']].dropna()

# ✅ Convert categorical columns to string
data_subset['Meal Type'] = data_subset['Meal Type'].astype(str)
data_subset['Dessert'] = data_subset['Dessert'].astype(str)

# ✅ Convert multi-label categorical columns to lists
data_subset['Menu'] = data_subset['Menu'].astype(str).apply(lambda x: x.split(','))
data_subset['Event'] = data_subset['Event'].astype(str).apply(lambda x: x.split(','))

# ✅ Compute historical average leftovers per menu item
menu_avg_leftovers = data_subset.explode('Menu').groupby('Menu')['leftovers'].mean().to_dict()
data_subset['Menu Avg Leftovers'] = data_subset['Menu'].apply(
    lambda items: np.mean([menu_avg_leftovers.get(item, 0) for item in items])
)

# ✅ One-hot encode multi-label categorical variables
mlb_menu = MultiLabelBinarizer()
mlb_event = MultiLabelBinarizer()
menu_encoded = pd.DataFrame(mlb_menu.fit_transform(data_subset['Menu']), columns=mlb_menu.classes_)
event_encoded = pd.DataFrame(mlb_event.fit_transform(data_subset['Event']), columns=mlb_event.classes_)

# ✅ Concatenate encoded menu & event features with other categorical features
data_encoded = pd.concat([data_subset.drop(columns=['Menu', 'Event']), menu_encoded, event_encoded], axis=1)

# ✅ Define categorical columns
categorical_features = ['Meal Type', 'Dessert']

# ✅ Preprocessing pipeline: One-hot encode categorical variables
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('scale', StandardScaler(), ['Menu Avg Leftovers'])
    ],
    remainder='passthrough'
)

# ✅ Apply preprocessing
X = preprocessor.fit_transform(data_encoded.drop(columns=['leftovers']))

# ✅ Convert processed X into a DataFrame
X = pd.DataFrame(X, columns=preprocessor.get_feature_names_out())

# ✅ Define target variable (log-transformed leftovers)
y_total = np.log1p(data_encoded[['leftovers']])

# ✅ Train-test split for total leftovers prediction
X_train, X_test, y_train_total, y_test_total = train_test_split(X, y_total, test_size=0.2, random_state=42)

# ✅ Train a new Voting Regressor
voting_regressor = VotingRegressor(
    estimators=[
        ('XGBoost', XGBRegressor(n_estimators=300, learning_rate=0.01, max_depth=4, objective="reg:squarederror", random_state=42)),
        ('CatBoost', CatBoostRegressor(iterations=500, learning_rate=0.01, depth=6, verbose=0, random_state=42)),
        ('RandomForest', RandomForestRegressor(n_estimators=300, max_depth=20, min_samples_leaf=5, random_state=42))
    ]
)

voting_regressor.fit(X_train, y_train_total.values.ravel())
y_pred_total = np.expm1(voting_regressor.predict(X_test))

# ✅ Compute R² score for Voting Regressor
regressor_r2 = r2_score(y_test_total, np.log1p(y_pred_total))
print(f"Voting Regressor R²: {regressor_r2:.4f}")


Voting Regressor R²: 0.6179


In [ ]:
# Save the trained Voting Regressor
joblib.dump(voting_regressor, "/content/drive/MyDrive/육사 부식 잔반 최적화/voting_regressor.pkl")

# Save the preprocessor
joblib.dump(preprocessor, "/content/drive/MyDrive/육사 부식 잔반 최적화/preprocessor.pkl")

# Save MultiLabelBinarizers
joblib.dump(mlb_menu, "/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_menu.pkl")
joblib.dump(mlb_event, "/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_event.pkl")

print("All components saved successfully!")


All components saved successfully!


In [ ]:

# 🔄 Reload trained components
voting_regressor = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/voting_regressor.pkl")
preprocessor = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/preprocessor.pkl")
mlb_menu = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_menu.pkl")
mlb_event = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_event.pkl")
model = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/voting_regressor.pkl")


Matrix Factorization + NNLS

In [ ]:
# 고유 음식 종류 개수 출력
unique_items = set(item.strip() for sublist in data_subset['Menu'] for item in sublist)
print(f"총 고유 음식 종류 수: {len(unique_items)}")


총 고유 음식 종류 수: 694


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import nnls
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import joblib


def train_matrix_factorization(data_subset, mlb_menu, n_factors=700):  #n_factors 줄이면 더 그럴싸해보임
    """Train NMF with robust normalization to avoid division by zero"""
    historical_menu_encoded = mlb_menu.transform(data_subset['Menu'])

    # Ensure valid factorization dimensions
    n_samples, n_features = historical_menu_encoded.shape
    n_factors = min(n_factors, n_samples, n_features)

    nmf = NMF(
        n_components=n_factors,
        init='nndsvda',
        solver='mu',
        beta_loss='kullback-leibler',
        random_state=42
    )

    W = nmf.fit_transform(historical_menu_encoded)
    H = nmf.components_

    # Stable normalization with condition number check
    norms = np.linalg.norm(H, axis=0, keepdims=True)
    H_normalized = H / np.where(norms < 1e-10, 1e-10, norms)

    return nmf, H_normalized

# ================== NNLS Distribution ==================
def distribute_leftovers(total_leftovers, menu_encoded_values, H):
    """Distribute totals using NNLS with item interactions"""
    individual_preds = []

    for total, menu_vec in zip(total_leftovers, menu_encoded_values):
        active_idx = np.where(menu_vec > 0)[0]

        if not active_idx.size:
            individual_preds.append(np.zeros_like(menu_vec))
            continue

        # Use latent factors for active items
        H_sub = H[:, active_idx]
        A = H_sub.T @ H_sub + 1e-6 * np.eye(H_sub.shape[1])  # Regularization
        b = H_sub.T @ np.ones(H_sub.shape[0]) * total

        weights, _ = nnls(A, b)

        # Force sum to match total leftovers exactly
        weights /= weights.sum() + 1e-10  # Normalize weights to sum to 1
        weights *= total                 # Scale weights to match total

        full_weights = np.zeros_like(menu_vec)
        full_weights[active_idx] = weights

        individual_preds.append(full_weights)

    return np.array(individual_preds)

def predict_new_meals(new_meals):

    def predict_total_leftovers(new_meals):
      """
      Predicts total leftovers for new meal combinations using the Voting Regressor.
      Handles unseen combinations of known menu items.
      """
      # Preprocess new data to match training structure
      processed = new_meals.copy()
      processed["Menu"] = processed["Menu"].str.split(",")
      processed["Event"] = processed["Event"].str.split(",")

      # Filter to known menu/event items from training data
      valid_menu = set(mlb_menu.classes_)
      valid_events = set(mlb_event.classes_)
      processed["Menu"] = processed["Menu"].apply(
          lambda x: [item.strip() for item in x if item.strip() in valid_menu]
      )
      processed["Event"] = processed["Event"].apply(
          lambda x: [item.strip() for item in x if item.strip() in valid_events]
      )

      # Compute "Menu Avg Leftovers" using historical training averages
      processed["Menu Avg Leftovers"] = processed["Menu"].apply(
          lambda items: np.mean([menu_avg_leftovers.get(item, 0) for item in items])
      )

      # Encode menu and event with MultiLabelBinarizer (ensure all training columns exist)
      menu_encoded = pd.DataFrame(
          mlb_menu.transform(processed["Menu"]),
          columns=mlb_menu.classes_,
          index=processed.index
      ).reindex(columns=mlb_menu.classes_, fill_value=0)  # Force all training columns

      event_encoded = pd.DataFrame(
          mlb_event.transform(processed["Event"]),
          columns=mlb_event.classes_,
          index=processed.index
      ).reindex(columns=mlb_event.classes_, fill_value=0)  # Force all training columns

      # Prepare raw features for ColumnTransformer
      features = pd.concat([
          processed[["Meal Type", "Dessert", "Menu Avg Leftovers"]],
          menu_encoded,
          event_encoded
      ], axis=1)

      # Apply the preprocessor (includes OneHotEncoder for Meal Type/Dessert)
      aligned_features = preprocessor.transform(features)

      # Predict totals
      totals = np.expm1(voting_regressor.predict(aligned_features))

      return totals

    """Predict total and distribute to items without historical averages"""
    # Load components
    model = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/voting_regressor.pkl")
    preprocessor = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/preprocessor.pkl")
    mlb_menu = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_menu.pkl")
    mlb_event = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_event.pkl")
    # Preprocessing
    processed = new_meals.copy()
    processed["Menu"] = processed["Menu"].str.split(",")
    processed["Event"] = processed["Event"].str.split(",")

    # Keep all input items (even unseen ones)
    all_items = list(set(item for sublist in processed["Menu"] for item in sublist))

    # Encode features
    menu_encoded = pd.DataFrame(
        mlb_menu.transform(processed["Menu"]),
        columns=mlb_menu.classes_,
        index=processed.index
    ).reindex(columns=all_items, fill_value=0)

    event_encoded = pd.DataFrame(
        mlb_event.transform(processed["Event"]),
        columns=mlb_event.classes_,
        index=processed.index
    )

    # Prepare features
    features = pd.concat([
        processed[["Meal Type", "Dessert"]],
        menu_encoded,
        event_encoded
    ], axis=1)

    # Align columns
    aligned_features = features.reindex(columns=preprocessor.get_feature_names_out(), fill_value=0)

    # Predict totals
    totals = predict_total_leftovers(new_meals)

        # After preprocessing in predict_new_meals():
    #print("Aligned Features Sample:\n", aligned_features.head())
    #print("Feature Means:\n", aligned_features.mean(axis=0))


    # Distribute using NNLS
    nmf, H = train_matrix_factorization(data_subset, mlb_menu)
    individual_preds = distribute_leftovers(totals, menu_encoded.values, H)
    # Save NMF model
    joblib.dump(nmf, "/content/drive/MyDrive/육사 부식 잔반 최적화/nmf_model.pkl")

    # Also save the item factors matrix H
    joblib.dump(nmf.components_, "/content/drive/MyDrive/육사 부식 잔반 최적화/nmf_components.pkl")

    # Add return statement
    return pd.DataFrame(individual_preds, columns=all_items), totals





In [ ]:
# 🧪 Example Usage
test_meals = pd.DataFrame([
    {
        "Meal Type": "B",
        "Menu": "베이컨불고기치즈버거,닭다리후라이드,치킨무나쵸소스,푸실리샐러드,시리얼",
        "Dessert": "1",
        "Event": "3"
    }
])
individual, total = predict_new_meals(test_meals)
print(f"Total Prediction: {total[0]:.2f}")
print("Individual Contributions:")
print(individual.loc[:, individual.iloc[0] > 0])

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Total Prediction: 83.12
Individual Contributions:
   베이컨불고기치즈버거  닭다리후라이드  푸실리샐러드  시리얼  치킨무나쵸소스
0          20       14      14   19       14


Error Bound

In [ ]:
nmf = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/nmf_model.pkl")

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import nnls
import joblib

# Load all components
nmf = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/nmf_model.pkl")
voting_regressor = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/voting_regressor.pkl")
preprocessor = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/preprocessor.pkl")
mlb_menu = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_menu.pkl")
mlb_event = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_event.pkl")

# Load dataset with true leftovers
data = pd.read_excel("/content/drive/MyDrive/육사 부식 잔반 최적화/current_processed_menu_data.xlsx")
data["Menu"] = data["Menu"].astype(str).apply(lambda x: sorted([item.strip() for item in x.split(",")]))
data["Event"] = data["Event"].astype(str).apply(lambda x: sorted([item.strip() for item in x.split(",")]))


def predict_total_leftovers(new_meals):
    processed = new_meals.copy()
    processed["Menu"] = processed["Menu"].str.split(",").apply(lambda x: [i.strip() for i in x])
    processed["Event"] = processed["Event"].str.split(",").apply(lambda x: [i.strip() for i in x])

    # Compute Menu Avg Leftovers from historical dictionary
    menu_avg_leftovers = data.explode("Menu").groupby("Menu")["leftovers"].mean().to_dict()
    processed["Menu Avg Leftovers"] = processed["Menu"].apply(
        lambda items: np.mean([menu_avg_leftovers.get(item, 0) for item in items])
    )

    # Encoding
    menu_encoded = pd.DataFrame(
        mlb_menu.transform(processed["Menu"]),
        columns=mlb_menu.classes_,
        index=processed.index
    )
    event_encoded = pd.DataFrame(
        mlb_event.transform(processed["Event"]),
        columns=mlb_event.classes_,
        index=processed.index
    )

    # Concatenate features
    features = pd.concat([
        processed[["Meal Type", "Dessert", "Menu Avg Leftovers"]],
        menu_encoded,
        event_encoded
    ], axis=1)

    X_input = preprocessor.transform(features)
    total_preds = np.expm1(voting_regressor.predict(X_input))

    return total_preds[0], menu_encoded, processed


def predict_with_bounds_and_theory(new_meals):
    total_pred, menu_encoded, processed = predict_total_leftovers(new_meals)
    menu_vec = menu_encoded.values[0]
    H = nmf.components_
    active_idx = np.where(menu_vec > 0)[0]
    if not active_idx.size:
        return pd.DataFrame()

    H_active = H[:, active_idx]  # shape: (r, |active|)
    cond_H = np.linalg.cond(H_active)  # from paper: κ(H)

    # Compute NNLS weights
    A = H_active.T @ H_active + 1e-6 * np.eye(len(active_idx))
    b = H_active.T @ np.ones(H_active.shape[0]) * total_pred
    weights, _ = nnls(A, b)
    weights = weights / (weights.sum() + 1e-10) * total_pred

    # Get total_true by matching record
    matched = data[
        (data["Meal Type"] == processed["Meal Type"].iloc[0]) &
        (data["Dessert"] == processed["Dessert"].iloc[0]) &
        (data["Menu"].apply(lambda x: sorted(x) == sorted(processed["Menu"].iloc[0]))) &
        (data["Event"].apply(lambda x: sorted(x) == sorted(processed["Event"].iloc[0])))
    ]
    if not matched.empty:
        total_true = matched["leftovers"].values[0]
    else:
        total_true = total_pred  # fallback: assume perfect match

    delta_T = abs(total_pred - total_true)

    # Estimate e_i from full reconstruction error
    V = mlb_menu.transform(data["Menu"])
    W = nmf.transform(V)
    V_hat = W @ H
    #recon_error_vector = np.abs(V - V_hat).mean(axis=0)  # per-item NMF error
    recon_error_vector = np.abs(V - V_hat).max(axis=0)  # per-item NMF error

    # Final bound from theory: |x_i - x*_i| <= κ(H) * |T_hat - T*| + |e_i|
    results = []
    for i, idx in enumerate(active_idx):
        pred = weights[i]
        e_i = recon_error_vector[idx]
        bound = cond_H * delta_T + e_i
        results.append({
            "item": mlb_menu.classes_[idx],
            "prediction": pred,
            "margin": bound,
            "percent_error": (bound / (pred + 1e-8)) * 100
        })

    return pd.DataFrame(results)




In [ ]:
# ✅ Example usage
test_meals = pd.DataFrame([{
    "Meal Type": "D",
    "Menu": "영양밥,콩나물국,비엔나소시지야채볶음,돼지고기감자조림",
    "Dessert": "0",
    "Event": "3"
}])

result_df = predict_with_bounds_and_theory(test_meals)
print(result_df)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


         item  prediction    margin  percent_error
0    돼지고기감자조림   56.617107  0.545452       0.963404
1  비엔나소시지야채볶음   20.511335  0.500000       2.437677
2         영양밥   10.086863  0.000068       0.000676
3        콩나물국   29.822491  0.500930       1.679705


Program

1. 맨 위 library install 하기
2. 세션 재시작하기

In [ ]:
%%writefile predict_leftovers_app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from scipy.optimize import nnls

# Load models and encoders
best_pipeline = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/final_pipeline.pkl")
menu_avg_leftovers = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/menu_avg_leftovers.pkl")
mlb_menu = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_menu.pkl")
mlb_event = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_event.pkl")
nmf = joblib.load("/content/drive/MyDrive/육사 부식 잔반 최적화/nmf_model.pkl")
data = pd.read_excel("/content/drive/MyDrive/육사 부식 잔반 최적화/current_processed_menu_data.xlsx")
data["Menu"] = data["Menu"].astype(str).apply(lambda x: sorted([i.strip() for i in x.split(",")]))
data["Event"] = data["Event"].astype(str).apply(lambda x: sorted([i.strip() for i in x.split(",")]))

DESSERT_MAP = {"없음": "0", "유제품": "1", "과일": "2", "과일푸딩": "3", "이온음료/ 에이드/ 탄산": "4", "핫바": "5", "마카롱/ 초콜릿/ 에너지바": "6"}
EVENT_MAP = {"주말, 공휴일": "1", "주중": "0", "유격": "4", "중대 전술훈련 및 기본 훈련": "3"}
MEAL_TYPE_MAP = {"아침": "A", "점심": "B", "저녁": "C", "브런치": "D"}

def predict_leftovers(meal_type, menu_items, dessert, event):
    menu_items_list = [i.strip() for i in menu_items.split(",")]
    known_menu_items = [item for item in menu_items_list if item in menu_avg_leftovers]
    if not known_menu_items:
        st.error("Error: Unknown menu items.")
        return {}

    avg_leftovers = np.mean([menu_avg_leftovers[item] for item in known_menu_items])
    raw_input = pd.DataFrame({
        'Meal Type': [meal_type],
        'Dessert': [dessert],
        'Menu Avg Leftovers': [avg_leftovers]
    })

    menu_encoded = pd.DataFrame(mlb_menu.transform([menu_items_list]), columns=mlb_menu.classes_)
    event_encoded = pd.DataFrame(mlb_event.transform([[event]]), columns=mlb_event.classes_)
    input_df = pd.concat([raw_input, menu_encoded, event_encoded], axis=1)
    input_np = best_pipeline.named_steps['preprocessor'].transform(input_df)

    total_pred = best_pipeline.named_steps['regressor'].predict(input_np)[0]
    menu_vec = menu_encoded.values[0]
    H = nmf.components_
    active_idx = np.where(menu_vec > 0)[0]

    H_active = H[:, active_idx]
    cond_H = np.linalg.cond(H_active)
    A = H_active.T @ H_active + 1e-6 * np.eye(len(active_idx))
    b = H_active.T @ np.ones(H_active.shape[0]) * total_pred
    weights, _ = nnls(A, b)
    weights = weights / (weights.sum() + 1e-10) * total_pred

    matched = data[(data["Meal Type"] == meal_type) &
                   (data["Dessert"] == dessert) &
                   (data["Menu"].apply(lambda x: sorted(x) == sorted(menu_items_list))) &
                   (data["Event"].apply(lambda x: sorted(x) == sorted([event])))]

    total_true = matched["leftovers"].values[0] if not matched.empty else total_pred
    delta_T = abs(total_pred - total_true)

    V = mlb_menu.transform(data["Menu"])
    W = nmf.transform(V)
    V_hat = W @ H
    recon_error_vector = np.abs(V - V_hat).max(axis=0)

    predictions = {}
    for i, idx in enumerate(active_idx):
        item = mlb_menu.classes_[idx]
        pred = weights[i]
        e_i = recon_error_vector[idx]
        bound = cond_H * delta_T + e_i

        # ✅ bound가 NaN 또는 inf이면 0으로 고정
        if not np.isfinite(bound):
            bound = 0.0

        percent = (bound / (pred + 1e-8)) * 100
        if not np.isfinite(percent) or percent > 100:
            percent = 0.0

        predictions[item] = f"{pred:.1f} ± {bound:.1f} ({percent:.1f}%)"



    return predictions

# 🎨 Streamlit UI with Korean Labels
st.title("🍛 군 급식 잔반 예측 프로그램")
st.markdown("메뉴 정보를 입력하면 예상 잔반량을 예측합니다.")

meal_type_korean = st.selectbox("🍽️ 식사 종류 선택", list(MEAL_TYPE_MAP.keys()))
meal_type = MEAL_TYPE_MAP[meal_type_korean]

menu_items = st.text_input("🍲 메뉴 항목 입력 (쉼표로 구분)", "영양밥,콩나물국,비엔나소시지야채볶음,돼지고기감자조림")

dessert_korean = st.selectbox("🍰 디저트 선택", list(DESSERT_MAP.keys()))
dessert = DESSERT_MAP[dessert_korean]

event_korean = st.selectbox("🎯 행사 선택", list(EVENT_MAP.keys()))
event = EVENT_MAP[event_korean]

# ✅ NEW: Enter number of people
num_people = st.number_input("👥 식사 인원 수", min_value=1, value=100)


# ✅ Predict button
if st.button("🧮 예측하기"):
    with st.spinner("계산 중..."):
        predictions = predict_leftovers(meal_type, menu_items, dessert, event)
        if predictions:
            scaled_predictions = {
                k: f"{float(v.split()[0]) * num_people / 1000:.2f} kg ± {float(v.split()[2]) * num_people / 1000:.2f} kg {v.split()[3].replace('((', '(').replace('))', ')')}"
                for k, v in predictions.items()
            }
            st.success("✅ 예측 완료!")
            st.write("### 🍽️ 예상 잔반량 (각 메뉴별)")
            st.json(scaled_predictions)

# 🔧 Additional Percentage Slider and Button
st.markdown("---")
st.subheader("🔧 특정 비율로 잔반량 계산")

percentage = st.slider("🔧 예측 잔반의 몇 퍼센트를 반환할까요?", min_value=1, max_value=100, value=50, step=1)

if st.button("🔄 특정 비율로 잔반 계산하기"):
    with st.spinner("계산 중..."):
        predictions = predict_leftovers(meal_type, menu_items, dessert, event)
        if predictions:
            scaled_predictions = {
                k: f"{float(v.split()[0]) * num_people * (percentage / 100) / 1000:.2f} kg ± {float(v.split()[2]) * num_people * (percentage / 100) / 1000:.2f} kg {v.split()[3].replace('((', '(').replace('))', ')')}"
                for k, v in predictions.items()
            }
            st.success(f"✅ 예측 완료! ({percentage}% 기준)")
            st.write(f"### 🍽️ 예상 잔반량 - {percentage}% 기준 (각 메뉴별)")
            st.json(scaled_predictions)




Overwriting predict_leftovers_app.py


In [ ]:
!nohup streamlit run predict_leftovers_app.py --server.port 8501 &
!ngrok config add-authtoken 2uO7qKTmyrri0YTq05KzyH0BWBW_7RhnrTVfK3JfGVzfHyCCq


nohup: appending output to 'nohup.out'
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Kill any previous ngrok processes
ngrok.kill()

# Start a new ngrok tunnel (fixing the API format issue)
public_url = ngrok.connect(8501, "http")  # ✅ FIX: Use `http` instead of `port`

print("🚀 Streamlit App is running at:", public_url)


🚀 Streamlit App is running at: NgrokTunnel: "https://0db1-35-236-163-7.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install pyinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.0/721.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.3/434.3 kB 14.0 MB/s eta 0:00:00


In [ ]:
import shutil
import os
from zipfile import ZipFile

# 압축할 폴더 생성
os.makedirs("/content/offline_app", exist_ok=True)

# ✅ Streamlit 앱 코드 저장
with open("/content/offline_app/predict_leftovers_app.py", "w", encoding="utf-8") as f:
    f.write(

"""
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from scipy.optimize import nnls

# Load models and encoders
best_pipeline = joblib.load("final_pipeline.pkl")
menu_avg_leftovers = joblib.load("menu_avg_leftovers.pkl")
mlb_menu = joblib.load("mlb_menu.pkl")
mlb_event = joblib.load("mlb_event.pkl")
nmf = joblib.load("nmf_model.pkl")
data = pd.read_excel("current_processed_menu_data.xlsx")
data["Menu"] = data["Menu"].astype(str).apply(lambda x: sorted([i.strip() for i in x.split(",")]))
data["Event"] = data["Event"].astype(str).apply(lambda x: sorted([i.strip() for i in x.split(",")]))

DESSERT_MAP = {"없음": "0", "유제품": "1", "과일": "2", "과일푸딩": "3", "이온음료/ 에이드/ 탄산": "4", "핫바": "5", "마카롱/ 초콜릿/ 에너지바": "6"}
EVENT_MAP = {"주말, 공휴일": "1", "주중": "0", "유격": "4", "중대 전술훈련 및 기본 훈련": "3"}
MEAL_TYPE_MAP = {"아침": "A", "점심": "B", "저녁": "C", "브런치": "D"}

def predict_leftovers(meal_type, menu_items, dessert, event):
    menu_items_list = [i.strip() for i in menu_items.split(",")]
    known_menu_items = [item for item in menu_items_list if item in menu_avg_leftovers]
    if not known_menu_items:
        st.error("Error: Unknown menu items.")
        return {}

    avg_leftovers = np.mean([menu_avg_leftovers[item] for item in known_menu_items])
    raw_input = pd.DataFrame({
        'Meal Type': [meal_type],
        'Dessert': [dessert],
        'Menu Avg Leftovers': [avg_leftovers]
    })

    menu_encoded = pd.DataFrame(mlb_menu.transform([menu_items_list]), columns=mlb_menu.classes_)
    event_encoded = pd.DataFrame(mlb_event.transform([[event]]), columns=mlb_event.classes_)
    input_df = pd.concat([raw_input, menu_encoded, event_encoded], axis=1)
    input_np = best_pipeline.named_steps['preprocessor'].transform(input_df)

    total_pred = best_pipeline.named_steps['regressor'].predict(input_np)[0]
    menu_vec = menu_encoded.values[0]
    H = nmf.components_
    active_idx = np.where(menu_vec > 0)[0]

    H_active = H[:, active_idx]
    cond_H = np.linalg.cond(H_active)
    A = H_active.T @ H_active + 1e-6 * np.eye(len(active_idx))
    b = H_active.T @ np.ones(H_active.shape[0]) * total_pred
    weights, _ = nnls(A, b)
    weights = weights / (weights.sum() + 1e-10) * total_pred

    matched = data[(data["Meal Type"] == meal_type) &
                   (data["Dessert"] == dessert) &
                   (data["Menu"].apply(lambda x: sorted(x) == sorted(menu_items_list))) &
                   (data["Event"].apply(lambda x: sorted(x) == sorted([event])))]

    total_true = matched["leftovers"].values[0] if not matched.empty else total_pred
    delta_T = abs(total_pred - total_true)

    V = mlb_menu.transform(data["Menu"])
    W = nmf.transform(V)
    V_hat = W @ H
    recon_error_vector = np.abs(V - V_hat).max(axis=0)

    predictions = {}
    for i, idx in enumerate(active_idx):
        item = mlb_menu.classes_[idx]
        pred = weights[i]
        e_i = recon_error_vector[idx]
        bound = cond_H * delta_T + e_i

        # ✅ bound가 NaN 또는 inf이면 0으로 고정
        if not np.isfinite(bound):
            bound = 0.0

        percent = (bound / (pred + 1e-8)) * 100
        if not np.isfinite(percent) or percent > 100:
            percent = 0.0

        predictions[item] = f"{pred:.1f} ± {bound:.1f} ({percent:.1f}%)"



    return predictions

# 🎨 Streamlit UI with Korean Labels
st.title("🍛 군 급식 잔반 예측 프로그램")
st.markdown("메뉴 정보를 입력하면 예상 잔반량을 예측합니다.")

meal_type_korean = st.selectbox("🍽️ 식사 종류 선택", list(MEAL_TYPE_MAP.keys()))
meal_type = MEAL_TYPE_MAP[meal_type_korean]

menu_items = st.text_input("🍲 메뉴 항목 입력 (쉼표로 구분)", "영양밥,콩나물국,비엔나소시지야채볶음,돼지고기감자조림")

dessert_korean = st.selectbox("🍰 디저트 선택", list(DESSERT_MAP.keys()))
dessert = DESSERT_MAP[dessert_korean]

event_korean = st.selectbox("🎯 행사 선택", list(EVENT_MAP.keys()))
event = EVENT_MAP[event_korean]

# ✅ NEW: Enter number of people
num_people = st.number_input("👥 식사 인원 수", min_value=1, value=100)


# ✅ Predict button
if st.button("🧮 예측하기"):
    with st.spinner("계산 중..."):
        predictions = predict_leftovers(meal_type, menu_items, dessert, event)
        if predictions:
            scaled_predictions = {
                k: f"{float(v.split()[0]) * num_people / 1000:.2f} kg ± {float(v.split()[2]) * num_people / 1000:.2f} kg {v.split()[3].replace('((', '(').replace('))', ')')}"
                for k, v in predictions.items()
            }
            st.success("✅ 예측 완료!")
            st.write("### 🍽️ 예상 잔반량 (각 메뉴별)")
            st.json(scaled_predictions)

# 🔧 Additional Percentage Slider and Button
st.markdown("---")
st.subheader("🔧 특정 비율로 잔반량 계산")

percentage = st.slider("🔧 예측 잔반의 몇 퍼센트를 반환할까요?", min_value=1, max_value=100, value=50, step=1)

if st.button("🔄 특정 비율로 잔반 계산하기"):
    with st.spinner("계산 중..."):
        predictions = predict_leftovers(meal_type, menu_items, dessert, event)
        if predictions:
            scaled_predictions = {
                k: f"{float(v.split()[0]) * num_people * (percentage / 100) / 1000:.2f} kg ± {float(v.split()[2]) * num_people * (percentage / 100) / 1000:.2f} kg {v.split()[3].replace('((', '(').replace('))', ')')}"
                for k, v in predictions.items()
            }
            st.success(f"✅ 예측 완료! ({percentage}% 기준)")
            st.write(f"### 🍽️ 예상 잔반량 - {percentage}% 기준 (각 메뉴별)")
            st.json(scaled_predictions)

""")

# ✅ requirements.txt 생성
with open("/content/offline_app/requirements.txt", "w") as f:
    f.write("""
streamlit==1.32.2
scikit-learn==1.4.2
numpy==1.26.4
pandas==2.2.2
joblib==1.4.0
openpyxl==3.1.2
scipy==1.13.0
xgboost==2.0.3
catboost==1.2.7
""")


# ✅ 모델과 데이터 복사
file_paths = [
    "/content/drive/MyDrive/육사 부식 잔반 최적화/final_pipeline.pkl",
    "/content/drive/MyDrive/육사 부식 잔반 최적화/menu_avg_leftovers.pkl",
    "/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_menu.pkl",
    "/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_event.pkl",
    "/content/drive/MyDrive/육사 부식 잔반 최적화/nmf_model.pkl",
    "/content/drive/MyDrive/육사 부식 잔반 최적화/current_processed_menu_data.xlsx",
]

for path in file_paths:
    shutil.copy(path, "/content/offline_app")

# ✅ 압축하기
with ZipFile("/content/predict_leftovers_offline_package.zip", "w") as zipf:
    for fname in os.listdir("/content/offline_app"):
        zipf.write(os.path.join("/content/offline_app", fname), arcname=fname)


In [ ]:
!zip -r /content/offline_app.zip /content/offline_app
from google.colab import files
files.download("/content/offline_app.zip")


updating: content/offline_app/ (stored 0%)
updating: content/offline_app/mlb_menu.pkl (deflated 63%)
updating: content/offline_app/requirements.txt (deflated 24%)
updating: content/offline_app/predict_leftovers_app.py (deflated 60%)
updating: content/offline_app/final_pipeline.pkl (deflated 76%)
updating: content/offline_app/current_processed_menu_data.xlsx (deflated 8%)
updating: content/offline_app/mlb_event.pkl (deflated 29%)
updating: content/offline_app/nmf_model.pkl (deflated 99%)
updating: content/offline_app/menu_avg_leftovers.pkl (deflated 66%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from scipy.optimize import nnls

# Load models and encoders
best_pipeline = joblib.load("final_pipeline.pkl")
menu_avg_leftovers = joblib.load("menu_avg_leftovers.pkl")
mlb_menu = joblib.load("mlb_menu.pkl")
mlb_event = joblib.load("mlb_event.pkl")
nmf = joblib.load("nmf_model.pkl")
data = pd.read_excel("current_processed_menu_data.xlsx")
data["Menu"] = data["Menu"].astype(str).apply(lambda x: sorted([i.strip() for i in x.split(",")]))
data["Event"] = data["Event"].astype(str).apply(lambda x: sorted([i.strip() for i in x.split(",")]))

DESSERT_MAP = {"없음": "0", "유제품": "1", "과일": "2", "과일푸딩": "3", "이온음료/ 에이드/ 탄산": "4", "핫바": "5", "마카롱/ 초콜릿/ 에너지바": "6"}
EVENT_MAP = {"주말, 공휴일": "1", "주중": "0", "유격": "4", "중대 전술훈련 및 기본 훈련": "3"}
MEAL_TYPE_MAP = {"아침": "A", "점심": "B", "저녁": "C", "브런치": "D"}

def predict_leftovers(meal_type, menu_items, dessert, event):
    menu_items_list = [i.strip() for i in menu_items.split(",")]
    known_menu_items = [item for item in menu_items_list if item in menu_avg_leftovers]
    if not known_menu_items:
        st.error("Error: Unknown menu items.")
        return {}

    avg_leftovers = np.mean([menu_avg_leftovers[item] for item in known_menu_items])
    raw_input = pd.DataFrame({
        'Meal Type': [meal_type],
        'Dessert': [dessert],
        'Menu Avg Leftovers': [avg_leftovers]
    })

    menu_encoded = pd.DataFrame(mlb_menu.transform([menu_items_list]), columns=mlb_menu.classes_)
    event_encoded = pd.DataFrame(mlb_event.transform([[event]]), columns=mlb_event.classes_)
    input_df = pd.concat([raw_input, menu_encoded, event_encoded], axis=1)
    input_np = best_pipeline.named_steps['preprocessor'].transform(input_df)

    total_pred = best_pipeline.named_steps['regressor'].predict(input_np)[0]
    menu_vec = menu_encoded.values[0]
    H = nmf.components_
    active_idx = np.where(menu_vec > 0)[0]

    H_active = H[:, active_idx]
    cond_H = np.linalg.cond(H_active)
    A = H_active.T @ H_active + 1e-6 * np.eye(len(active_idx))
    b = H_active.T @ np.ones(H_active.shape[0]) * total_pred
    weights, _ = nnls(A, b)
    weights = weights / (weights.sum() + 1e-10) * total_pred

    matched = data[(data["Meal Type"] == meal_type) &
                   (data["Dessert"] == dessert) &
                   (data["Menu"].apply(lambda x: sorted(x) == sorted(menu_items_list))) &
                   (data["Event"].apply(lambda x: sorted(x) == sorted([event])))]

    total_true = matched["leftovers"].values[0] if not matched.empty else total_pred
    delta_T = abs(total_pred - total_true)

    V = mlb_menu.transform(data["Menu"])
    W = nmf.transform(V)
    V_hat = W @ H
    recon_error_vector = np.abs(V - V_hat).max(axis=0)

    predictions = {}
    for i, idx in enumerate(active_idx):
        item = mlb_menu.classes_[idx]
        pred = weights[i]
        e_i = recon_error_vector[idx]
        bound = cond_H * delta_T + e_i

        # ✅ bound가 NaN 또는 inf이면 0으로 고정
        if not np.isfinite(bound):
            bound = 0.0

        percent = (bound / (pred + 1e-8)) * 100
        if not np.isfinite(percent) or percent > 100:
            percent = 0.0

        predictions[item] = f"{pred:.1f} ± {bound:.1f} ({percent:.1f}%)"



    return predictions

# 🎨 Streamlit UI with Korean Labels
st.title("🍛 군 급식 잔반 예측 프로그램")
st.markdown("메뉴 정보를 입력하면 예상 잔반량을 예측합니다.")

meal_type_korean = st.selectbox("🍽️ 식사 종류 선택", list(MEAL_TYPE_MAP.keys()))
meal_type = MEAL_TYPE_MAP[meal_type_korean]

menu_items = st.text_input("🍲 메뉴 항목 입력 (쉼표로 구분)", "영양밥,콩나물국,비엔나소시지야채볶음,돼지고기감자조림")

dessert_korean = st.selectbox("🍰 디저트 선택", list(DESSERT_MAP.keys()))
dessert = DESSERT_MAP[dessert_korean]

event_korean = st.selectbox("🎯 행사 선택", list(EVENT_MAP.keys()))
event = EVENT_MAP[event_korean]

# ✅ NEW: Enter number of people
num_people = st.number_input("👥 식사 인원 수", min_value=1, value=100)


# ✅ Predict button
if st.button("🧮 예측하기"):
    with st.spinner("계산 중..."):
        predictions = predict_leftovers(meal_type, menu_items, dessert, event)
        if predictions:
            scaled_predictions = {
                k: f"{float(v.split()[0]) * num_people / 1000:.2f} kg ± {float(v.split()[2]) * num_people / 1000:.2f} kg {v.split()[3].replace('((', '(').replace('))', ')')}"
                for k, v in predictions.items()
            }
            st.success("✅ 예측 완료!")
            st.write("### 🍽️ 예상 잔반량 (각 메뉴별)")
            st.json(scaled_predictions)

# 🔧 Additional Percentage Slider and Button
st.markdown("---")
st.subheader("🔧 특정 비율로 잔반량 계산")

percentage = st.slider("🔧 예측 잔반의 몇 퍼센트를 반환할까요?", min_value=1, max_value=100, value=50, step=1)

if st.button("🔄 특정 비율로 잔반 계산하기"):
    with st.spinner("계산 중..."):
        predictions = predict_leftovers(meal_type, menu_items, dessert, event)
        if predictions:
            scaled_predictions = {
                k: f"{float(v.split()[0]) * num_people * (percentage / 100) / 1000:.2f} kg ± {float(v.split()[2]) * num_people * (percentage / 100) / 1000:.2f} kg {v.split()[3].replace('((', '(').replace('))', ')')}"
                for k, v in predictions.items()
            }
            st.success(f"✅ 예측 완료! ({percentage}% 기준)")
            st.write(f"### 🍽️ 예상 잔반량 - {percentage}% 기준 (각 메뉴별)")
            st.json(scaled_predictions)

In [ ]:
import os
import shutil
from zipfile import ZipFile

# 🔧 오프라인 앱 폴더 초기화
base_dir = "/content/offline_app"
os.makedirs(base_dir, exist_ok=True)
os.makedirs(f"{base_dir}/whl", exist_ok=True)

# ✅ 필요한 .whl 파일들 다운로드
wheels = {
    "numpy": "https://download.lfd.uci.edu/pythonlibs/w4tscw6k/numpy-1.26.4-cp311-cp311-win_amd64.whl",
    "pandas": "https://download.lfd.uci.edu/pythonlibs/w4tscw6k/pandas-2.2.2-cp311-cp311-win_amd64.whl",
    "scipy": "https://download.lfd.uci.edu/pythonlibs/w4tscw6k/scipy-1.13.0-cp311-cp311-win_amd64.whl",
    "scikit-learn": "https://download.lfd.uci.edu/pythonlibs/w4tscw6k/scikit_learn-1.4.2-cp311-cp311-win_amd64.whl",
    "catboost": "https://download.lfd.uci.edu/pythonlibs/w4tscw6k/catboost-1.2.7-cp311-cp311-win_amd64.whl",
    "xgboost": "https://download.lfd.uci.edu/pythonlibs/w4tscw6k/xgboost-2.0.3-cp311-cp311-win_amd64.whl",
    "openpyxl": "https://download.lfd.uci.edu/pythonlibs/w4tscw6k/openpyxl-3.1.2-py3-none-any.whl",
    "joblib": "https://download.lfd.uci.edu/pythonlibs/w4tscw6k/joblib-1.4.0-py3-none-any.whl",
    "streamlit": "https://files.pythonhosted.org/packages/7d/b6/f9e62e508caaa8b5c2c4f6aa4fc6469b1b30761c7fdc6027c07e55d7918d/streamlit-1.32.2-py2.py3-none-any.whl"
}

from urllib.request import urlretrieve

for name, url in wheels.items():
    out_path = os.path.join(base_dir, "whl", os.path.basename(url))
    if not os.path.exists(out_path):
        urlretrieve(url, out_path)

# ✅ requirements.txt 생성
with open(f"{base_dir}/requirements.txt", "w") as f:
    f.write("""numpy
pandas
scipy
scikit-learn
catboost
xgboost
openpyxl
joblib
streamlit""")

# ✅ start_app.bat 생성
with open(f"{base_dir}/start_app.bat", "w", encoding="utf-8") as f:
    f.write(r"""@echo off
cd /d "%~dp0"
python -m venv venv
call venv\Scripts\activate
pip install --upgrade pip
pip install --no-index --find-links=whl -r requirements.txt
streamlit run predict_leftovers_app.py
pause
""")

with open("/content/offline_app/predict_leftovers_app.py", "w", encoding="utf-8") as f:
    f.write(

"""
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from scipy.optimize import nnls

# Load models and encoders
best_pipeline = joblib.load("final_pipeline.pkl")
menu_avg_leftovers = joblib.load("menu_avg_leftovers.pkl")
mlb_menu = joblib.load("mlb_menu.pkl")
mlb_event = joblib.load("mlb_event.pkl")
nmf = joblib.load("nmf_model.pkl")
data = pd.read_excel("current_processed_menu_data.xlsx")
data["Menu"] = data["Menu"].astype(str).apply(lambda x: sorted([i.strip() for i in x.split(",")]))
data["Event"] = data["Event"].astype(str).apply(lambda x: sorted([i.strip() for i in x.split(",")]))

DESSERT_MAP = {"없음": "0", "유제품": "1", "과일": "2", "과일푸딩": "3", "이온음료/ 에이드/ 탄산": "4", "핫바": "5", "마카롱/ 초콜릿/ 에너지바": "6"}
EVENT_MAP = {"주말, 공휴일": "1", "주중": "0", "유격": "4", "중대 전술훈련 및 기본 훈련": "3"}
MEAL_TYPE_MAP = {"아침": "A", "점심": "B", "저녁": "C", "브런치": "D"}

def predict_leftovers(meal_type, menu_items, dessert, event):
    menu_items_list = [i.strip() for i in menu_items.split(",")]
    known_menu_items = [item for item in menu_items_list if item in menu_avg_leftovers]
    if not known_menu_items:
        st.error("Error: Unknown menu items.")
        return {}

    avg_leftovers = np.mean([menu_avg_leftovers[item] for item in known_menu_items])
    raw_input = pd.DataFrame({
        'Meal Type': [meal_type],
        'Dessert': [dessert],
        'Menu Avg Leftovers': [avg_leftovers]
    })

    menu_encoded = pd.DataFrame(mlb_menu.transform([menu_items_list]), columns=mlb_menu.classes_)
    event_encoded = pd.DataFrame(mlb_event.transform([[event]]), columns=mlb_event.classes_)
    input_df = pd.concat([raw_input, menu_encoded, event_encoded], axis=1)
    input_np = best_pipeline.named_steps['preprocessor'].transform(input_df)

    total_pred = best_pipeline.named_steps['regressor'].predict(input_np)[0]
    menu_vec = menu_encoded.values[0]
    H = nmf.components_
    active_idx = np.where(menu_vec > 0)[0]

    H_active = H[:, active_idx]
    cond_H = np.linalg.cond(H_active)
    A = H_active.T @ H_active + 1e-6 * np.eye(len(active_idx))
    b = H_active.T @ np.ones(H_active.shape[0]) * total_pred
    weights, _ = nnls(A, b)
    weights = weights / (weights.sum() + 1e-10) * total_pred

    matched = data[(data["Meal Type"] == meal_type) &
                   (data["Dessert"] == dessert) &
                   (data["Menu"].apply(lambda x: sorted(x) == sorted(menu_items_list))) &
                   (data["Event"].apply(lambda x: sorted(x) == sorted([event])))]

    total_true = matched["leftovers"].values[0] if not matched.empty else total_pred
    delta_T = abs(total_pred - total_true)

    V = mlb_menu.transform(data["Menu"])
    W = nmf.transform(V)
    V_hat = W @ H
    recon_error_vector = np.abs(V - V_hat).max(axis=0)

    predictions = {}
    for i, idx in enumerate(active_idx):
        item = mlb_menu.classes_[idx]
        pred = weights[i]
        e_i = recon_error_vector[idx]
        bound = cond_H * delta_T + e_i

        # ✅ bound가 NaN 또는 inf이면 0으로 고정
        if not np.isfinite(bound):
            bound = 0.0

        percent = (bound / (pred + 1e-8)) * 100
        if not np.isfinite(percent) or percent > 100:
            percent = 0.0

        predictions[item] = f"{pred:.1f} ± {bound:.1f} ({percent:.1f}%)"



    return predictions

# 🎨 Streamlit UI with Korean Labels
st.title("🍛 군 급식 잔반 예측 프로그램")
st.markdown("메뉴 정보를 입력하면 예상 잔반량을 예측합니다.")

meal_type_korean = st.selectbox("🍽️ 식사 종류 선택", list(MEAL_TYPE_MAP.keys()))
meal_type = MEAL_TYPE_MAP[meal_type_korean]

menu_items = st.text_input("🍲 메뉴 항목 입력 (쉼표로 구분)", "영양밥,콩나물국,비엔나소시지야채볶음,돼지고기감자조림")

dessert_korean = st.selectbox("🍰 디저트 선택", list(DESSERT_MAP.keys()))
dessert = DESSERT_MAP[dessert_korean]

event_korean = st.selectbox("🎯 행사 선택", list(EVENT_MAP.keys()))
event = EVENT_MAP[event_korean]

# ✅ NEW: Enter number of people
num_people = st.number_input("👥 식사 인원 수", min_value=1, value=100)


# ✅ Predict button
if st.button("🧮 예측하기"):
    with st.spinner("계산 중..."):
        predictions = predict_leftovers(meal_type, menu_items, dessert, event)
        if predictions:
            scaled_predictions = {
                k: f"{float(v.split()[0]) * num_people / 1000:.2f} kg ± {float(v.split()[2]) * num_people / 1000:.2f} kg {v.split()[3].replace('((', '(').replace('))', ')')}"
                for k, v in predictions.items()
            }
            st.success("✅ 예측 완료!")
            st.write("### 🍽️ 예상 잔반량 (각 메뉴별)")
            st.json(scaled_predictions)

# 🔧 Additional Percentage Slider and Button
st.markdown("---")
st.subheader("🔧 특정 비율로 잔반량 계산")

percentage = st.slider("🔧 예측 잔반의 몇 퍼센트를 반환할까요?", min_value=1, max_value=100, value=50, step=1)

if st.button("🔄 특정 비율로 잔반 계산하기"):
    with st.spinner("계산 중..."):
        predictions = predict_leftovers(meal_type, menu_items, dessert, event)
        if predictions:
            scaled_predictions = {
                k: f"{float(v.split()[0]) * num_people * (percentage / 100) / 1000:.2f} kg ± {float(v.split()[2]) * num_people * (percentage / 100) / 1000:.2f} kg {v.split()[3].replace('((', '(').replace('))', ')')}"
                for k, v in predictions.items()
            }
            st.success(f"✅ 예측 완료! ({percentage}% 기준)")
            st.write(f"### 🍽️ 예상 잔반량 - {percentage}% 기준 (각 메뉴별)")
            st.json(scaled_predictions)

""")

# ✅ 모델 및 데이터 파일 복사
file_paths = {
    "final_pipeline.pkl": "/content/drive/MyDrive/육사 부식 잔반 최적화/final_pipeline.pkl",
    "menu_avg_leftovers.pkl": "/content/drive/MyDrive/육사 부식 잔반 최적화/menu_avg_leftovers.pkl",
    "mlb_menu.pkl": "/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_menu.pkl",
    "mlb_event.pkl": "/content/drive/MyDrive/육사 부식 잔반 최적화/mlb_event.pkl",
    "nmf_model.pkl": "/content/drive/MyDrive/육사 부식 잔반 최적화/nmf_model.pkl",
    "current_processed_menu_data.xlsx": "/content/drive/MyDrive/육사 부식 잔반 최적화/current_processed_menu_data.xlsx",
}

for fname, origin in file_paths.items():
    shutil.copy(origin, os.path.join(base_dir, fname))

# ✅ 압축하기
zip_path = "/content/offline_app_full.zip"
with ZipFile(zip_path, "w") as zipf:
    for folder, _, files in os.walk(base_dir):
        for file in files:
            full_path = os.path.join(folder, file)
            rel_path = os.path.relpath(full_path, base_dir)
            zipf.write(full_path, arcname=rel_path)

# ✅ 다운로드 링크 제공
from google.colab import files
files.download(zip_path)


HTTPError: HTTP Error 404: Not Found